# Vehicle Registration Data Cleaning Steps

---


## 1. Import Required Libraries

Essential libraries for data manipulation and visualization.


In [141]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

## 2. Load Raw Vehicle Data

Read the raw vehicle registration data from the CSV file.


In [142]:
# Load the vehicle data CSV from the Dataset folder using a relative path
vehicle_df = pd.read_csv('../Dataset/Vehicle_Data.csv')


vehicle_df.tail()

,slno,registrationNo,regvalidfrom,regvalidto,makerName,modelDesc,bodyType,cc,cylinder,fuel,hp,seatCapacity,OfficeCd
1048570,14578590,TG08AA0036,30/04/2025,2040-04-29T16:41:10.347+05:30,TVS MOTOR COMPANY LTD,TVS XL100 HEAVY DUTY I-TOUCH START BSVI-PH2,Solo,100.0,1,PETROL,4.29,2,RTA MEDCHAL
1048571,14578591,GJ05BX2776,23/01/2017,2032-01-22T00:00:00+05:30,ASHOK LEYLAND LTD,ALPSV3/139 BSIV,Saloon,3840.0,4,DIESEL,127.39,26,RTA-HYDERABAD-SZ
1048572,14578592,MP09GG8143,1/5/2017,2032-04-30T00:00:00+05:30,ASHOK LEYLAND LTD,EC1214 15500G4200WBL7663DC5915X2280X603DSD BSIII,Open,3839.0,4,DIESEL,118.00,3,RTA-HYDERABAD-SZ
1048573,14578593,AP07TG9849,17/10/2016,2026-01-10T00:00:00+05:30,ASHOK LEYLAND LTD,2518T IL TIPPER BSIII,TIPPER,5760.0,6,DIESEL,177.01,2,RTA NALGONDA
1048574,14578594,PB65AU0368,20/12/2018,2033-12-19T00:00:00+05:30,ASHOK LEYLAND LTD,2518XL BSIV,Closed,5759.0,6,DIESEL,177.01,3,RTA-HYDERABAD-SZ


## 3. Initial Data Cleaning

Remove duplicate rows and empty columns to prepare for further cleaning.


In [143]:
# Remove duplicate rows based on all columns
vehicle_df.drop_duplicates(inplace=True)

# Drop columns that are completely empty
vehicle_df.dropna(axis=1, how='all', inplace=True)

print('Duplicates removed and empty columns dropped.')
print('Remaining columns:', vehicle_df.columns.tolist())

Duplicates removed and empty columns dropped.
Remaining columns: ['slno', 'registrationNo', 'regvalidfrom', 'regvalidto', 'makerName', 'modelDesc', 'bodyType', 'cc', 'cylinder', 'fuel', 'hp', 'seatCapacity', 'OfficeCd']


## 4. Data Preview

Preview the dataset and inspect key columns.


In [144]:
vehicle_df.head()

,slno,registrationNo,regvalidfrom,regvalidto,makerName,modelDesc,bodyType,cc,cylinder,fuel,hp,seatCapacity,OfficeCd
0,98,AP09TA3408,19/03/2009,2025-11-28T13:32:10.793+05:30,TATA MOTORS LIMITED,TATA ACE HT-2-SEATER-BS-III,Open,702.0,2,DIESEL,12.0,2,RTA RANGAREDDY
2,257,AP09TA3461,28/03/2009,2026-01-21T16:14:49.66+05:30,MARUTI UDYOG LTD,OMNI MPI CARGO BSIII,Saloon,796.0,3,PETROL,24.0,2,RTA MEDCHAL
3,350,AP09TA3987,24/07/2009,2026-02-27T15:16:50.093+05:30,TOYOTA KIRLOSKAR MOTOR,INNOVA 2.5L G,Saloon,2494.0,4,DIESEL,0.0,8,RTA WANAPARTHY
5,1266,AP09TA4149,2/4/2009,2026-01-16T15:39:07.347+05:30,EICHER MOTORS,10.90 G RHD E CAB & H.S.DECK,TANKER,3298.0,4,DIESEL,70.0,3,RTA RANGAREDDY
7,1268,TS35F8340,15/04/2009,2029-08-27T15:15:05.847+05:30,ESCORTS LIMITED,TATA SUMO VICTA DI CX 7 SEATER,Saloon,2956.0,4,DIESEL,52.0,7,RTA IBRAHIMPATNAM


## 5. Handle Missing Values & Normalize Fields

Fill missing values, normalize categorical fields, and standardize date columns.


In [145]:
# Inspect the data
print(vehicle_df.info())
print(vehicle_df.isnull().sum())

# Replace 'NA' and 'N/A' strings with np.nan for all columns
vehicle_df.replace(['NA', 'N/A'], np.nan, inplace=True)

# Handle missing values (fill with 'Unknown' for key fields, drop rows with missing registrationNo or regvalidfrom)
vehicle_df.fillna({'makerName': 'Unknown', 'modelDesc': 'Unknown', 'OfficeCd': 'Unknown'}, inplace=True)
vehicle_df.dropna(subset=['registrationNo', 'regvalidfrom'], inplace=True)

# Normalize categorical fields
vehicle_df['makerName'] = vehicle_df['makerName'].astype(str).str.strip().str.title()
vehicle_df['modelDesc'] = vehicle_df['modelDesc'].astype(str).str.strip().str.title()
vehicle_df['OfficeCd'] = vehicle_df['OfficeCd'].astype(str).str.strip().str.upper()

# Parse and standardize dates (remove time and timezone)
vehicle_df['regvalidfrom'] = pd.to_datetime(vehicle_df['regvalidfrom'], errors='coerce').dt.date
if 'regvalidto' in vehicle_df.columns:
    vehicle_df['regvalidto'] = pd.to_datetime(vehicle_df['regvalidto'], errors='coerce').dt.date
vehicle_df = vehicle_df[vehicle_df['regvalidfrom'].notnull()]

# Remove duplicates based on registrationNo
vehicle_df.drop_duplicates(subset=['registrationNo'], inplace=True)

# Validate data ranges (example: Year column)
if 'Year' in vehicle_df.columns:
    vehicle_df = vehicle_df[(vehicle_df['Year'] >= 1980) & (vehicle_df['Year'] <= 2025)]

# Save cleaned data (ensure output directory exists)
os.makedirs('../Dataset', exist_ok=True)
vehicle_df.to_csv('../Dataset/Vehicle_Data_Cleaned.csv', index=False)
print('Data cleaning complete. Cleaned file saved to ../Dataset/Vehicle_Data_Cleaned.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 540509 entries, 0 to 1048574
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   slno            540509 non-null  int64  
 1   registrationNo  537547 non-null  object 
 2   regvalidfrom    537600 non-null  object 
 3   regvalidto      540509 non-null  object 
 4   makerName       540509 non-null  object 
 5   modelDesc       540509 non-null  object 
 6   bodyType        540447 non-null  object 
 7   cc              540509 non-null  float64
 8   cylinder        540509 non-null  int64  
 9   fuel            533160 non-null  object 
 10  hp              540509 non-null  float64
 11  seatCapacity    540509 non-null  int64  
 12  OfficeCd        540509 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 57.7+ MB
None
slno                 0
registrationNo    2962
regvalidfrom      2909
regvalidto           0
makerName            0
modelDesc           

C:\Users\asus\AppData\Local\Temp\ipykernel_17608\1475618036.py:18: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  vehicle_df['regvalidfrom'] = pd.to_datetime(vehicle_df['regvalidfrom'], errors='coerce').dt.date


Data cleaning complete. Cleaned file saved to ../Dataset/Vehicle_Data_Cleaned.csv


## 6. Data Structure & Summary

Review the structure, data types, and summary statistics of the cleaned dataset.


In [146]:
# Understand the structure of the dataset
print('Shape of the dataset:', vehicle_df.shape)
print('Column names:', vehicle_df.columns.tolist())
print('Data types:')
print(vehicle_df.dtypes)
print('First 5 rows:')
print(vehicle_df.head())
print('Summary statistics:')
print(vehicle_df.describe(include='all'))

Shape of the dataset: (517812, 13)
Column names: ['slno', 'registrationNo', 'regvalidfrom', 'regvalidto', 'makerName', 'modelDesc', 'bodyType', 'cc', 'cylinder', 'fuel', 'hp', 'seatCapacity', 'OfficeCd']
Data types:
slno                int64
registrationNo     object
regvalidfrom       object
regvalidto         object
makerName          object
modelDesc          object
bodyType           object
cc                float64
cylinder            int64
fuel               object
hp                float64
seatCapacity        int64
OfficeCd           object
dtype: object
First 5 rows:
   slno registrationNo regvalidfrom  regvalidto               makerName  \
0    98     AP09TA3408   2009-03-19  2025-11-28     Tata Motors Limited   
2   257     AP09TA3461   2009-03-28  2026-01-21        Maruti Udyog Ltd   
3   350     AP09TA3987   2009-07-24  2026-02-27  Toyota Kirloskar Motor   
5  1266     AP09TA4149   2009-04-02  2026-01-16           Eicher Motors   
7  1268      TS35F8340   2009-04-15  2029-0

In [152]:
# Check max and min values in slno
if 'slno' in vehicle_df.columns:
    print("slno min:", vehicle_df['slno'].min())
    print("slno max:", vehicle_df['slno'].max())
    # Optionally, filter out rows with slno > 2_147_483_647 (SQL INT max)
    overflow_rows = vehicle_df[vehicle_df['slno'] > 2_147_483_647]
    if not overflow_rows.empty:
        print("Rows with slno overflow detected. Count:", len(overflow_rows))
        # Option 1: Remove them
        # vehicle_df = vehicle_df[vehicle_df['slno'] <= 2_147_483_647]
        # Option 2: Set to NaN or a valid value
        # vehicle_df.loc[vehicle_df['slno'] > 2_147_483_647, 'slno'] = np.nan
else:
    print("'slno' column not found.")

slno min: 98
slno max: 14578594


## 7. Final Normalization (If Needed)

Apply any final normalization steps to categorical fields if required.


In [147]:
# Fix KeyError: 'Model' by using correct column names
# The correct column for model is 'modelDesc', and for RTO use 'OfficeCd'

# Normalize categorical fields
vehicle_df['makerName'] = vehicle_df['makerName'].str.strip().str.title()
vehicle_df['modelDesc'] = vehicle_df['modelDesc'].str.strip().str.title()
vehicle_df['OfficeCd'] = vehicle_df['OfficeCd'].str.strip().str.upper()

vehicle_df 

print('Categorical fields normalized.')

Categorical fields normalized.


## 8. Convert year-only values in regvalidfrom and regvalidto to '01/01/YYYY' format


In [148]:
# Convert year-only values in regvalidfrom and regvalidto to '01/01/YYYY' format
def convert_year_to_date(val):
    if pd.isnull(val):
        return val
    # If value is an integer year or string year like '2024', '2019', etc.
    if re.fullmatch(r'20\d{2}', str(val).strip()):
        return f'01/01/{str(val).strip()}'
    return val
vehicle_df['regvalidfrom'] = vehicle_df['regvalidfrom'].apply(convert_year_to_date)
if 'regvalidto' in vehicle_df.columns:
    vehicle_df['regvalidto'] = vehicle_df['regvalidto'].apply(convert_year_to_date)
print('Year-only dates converted to 01/01/YYYY format in regvalidfrom and regvalidto.')

Year-only dates converted to 01/01/YYYY format in regvalidfrom and regvalidto.


## 9. Convert 'fuel' values of 0 or -1 to None (np.nan) or 'Unknown'

converting values like 0 or -1 -> none


In [149]:
if 'fuel' in vehicle_df.columns:
    # Define a small normalizer that handles numbers, numeric-strings, and string forms
    def normalize_fuel(val):
        # Keep existing missing values as np.nan
        if pd.isnull(val):
            return np.nan
        # Strings like 'NA'/'N/A' were already converted earlier to np.nan; handle numeric-like values
        s = str(val).strip()
        # direct string matches
        if s in ('0', '-1', '0.0', '-1.0'):
            return np.nan
        # try numeric conversion for mixed types (e.g., 0.0, 0)
        try:
            n = float(s)
            if n == 0.0 or n == -1.0:
                return np.nan
        except Exception:
            pass
        return val
    # Apply normalizer
    vehicle_df['fuel'] = vehicle_df['fuel'].apply(normalize_fuel)
    # Optionally convert column to a nullable dtype (keeps NaN)
    try:
        vehicle_df['fuel'] = vehicle_df['fuel'].astype('Float64')  # if fuel is numeric; preserves <NA> for pandas nullable float
    except Exception:
        # if mixed types (strings), keep as-is; NaNs remain as np.nan
        pass
    # Print a short summary of changes
    total = len(vehicle_df)
    nulls = vehicle_df['fuel'].isna().sum()
    print(f"Converted 0/-1 fuel values to NaN. fuel nulls: {nulls} / {total} rows.")
else:
    print("'fuel' column not found in dataframe.")

Converted 0/-1 fuel values to NaN. fuel nulls: 7837 / 517812 rows.


## 10. View Cleaned Data

Display the final cleaned dataset for review.


In [150]:
vehicle_df

,slno,registrationNo,regvalidfrom,regvalidto,makerName,modelDesc,bodyType,cc,cylinder,fuel,hp,seatCapacity,OfficeCd
0,98,AP09TA3408,2009-03-19,2025-11-28,Tata Motors Limited,Tata Ace Ht-2-Seater-Bs-Iii,Open,702.0,2,DIESEL,12.00,2,RTA RANGAREDDY
2,257,AP09TA3461,2009-03-28,2026-01-21,Maruti Udyog Ltd,Omni Mpi Cargo Bsiii,Saloon,796.0,3,PETROL,24.00,2,RTA MEDCHAL
3,350,AP09TA3987,2009-07-24,2026-02-27,Toyota Kirloskar Motor,Innova 2.5L G,Saloon,2494.0,4,DIESEL,0.00,8,RTA WANAPARTHY
5,1266,AP09TA4149,2009-04-02,2026-01-16,Eicher Motors,10.90 G Rhd E Cab & H.S.Deck,TANKER,3298.0,4,DIESEL,70.00,3,RTA RANGAREDDY
7,1268,TS35F8340,2009-04-15,2029-08-27,Escorts Limited,Tata Sumo Victa Di Cx 7 Seater,Saloon,2956.0,4,DIESEL,52.00,7,RTA IBRAHIMPATNAM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,14578590,TG08AA0036,2025-04-30,2040-04-29,Tvs Motor Company Ltd,Tvs Xl100 Heavy Duty I-Touch Start Bsvi-Ph2,Solo,100.0,1,PETROL,4.29,2,RTA MEDCHAL
1048571,14578591,GJ05BX2776,2017-01-23,NaT,Ashok Leyland Ltd,Alpsv3/139 Bsiv,Saloon,3840.0,4,DIESEL,127.39,26,RTA-HYDERABAD-SZ
1048572,14578592,MP09GG8143,2017-05-01,NaT,Ashok Leyland Ltd,Ec1214 15500G4200Wbl7663Dc5915X2280X603Dsd Bsiii,Open,3839.0,4,DIESEL,118.00,3,RTA-HYDERABAD-SZ
1048573,14578593,AP07TG9849,2016-10-17,NaT,Ashok Leyland Ltd,2518T Il Tipper Bsiii,TIPPER,5760.0,6,DIESEL,177.01,2,RTA NALGONDA


## 11. Exporting Cleaned Data

exporting the cleaned dataframe to a csv file


In [151]:
vehicle_df.to_csv('../Dataset/Vehicle_Data_Cleaned.csv', index=False)